In [1]:
import numpy as np

# using truncated multivariate normal
# https://stackoverflow.com/questions/20115917/truncated-multivariate-normal-in-scipy
import emcee

In [56]:
a = np.array([[1, -2j], [-2j, 5]])
w, v = eigh(a)

v.dot(v)



array([[ 0.85355339+0.14644661j,  0.35355339-0.35355339j],
       [ 0.35355339+0.35355339j, -0.85355339+0.14644661j]])

In [63]:
from numpy.linalg import inv, cholesky
from scipy.linalg import qr

def lnprob_trunc_norm(x, mean, bounds, C):
    if np.any(x < bounds[:,0]) or np.any(x > bounds[:,1]):
        return -np.inf
    else:
        return -0.5*(x-mean).dot(inv(C)).dot(x-mean)

In [125]:
def multivariate_trunc_normal(mean, cov, bounds, size=1):
    ndim = mean.shape[0]
    nwalkers = max(10*ndim, size)
    nsteps = 1000 + size
    
    S = emcee.EnsembleSampler(nwalkers, ndim, lnprob_trunc_norm, args = (mean, bounds, cov))
    pos = emcee.utils.sample_ball(mean, np.sqrt(np.diag(C)), size=nwalkers)
    pos, prob, state = S.run_mcmc(pos, nsteps)
    
    return pos[-size:].flatten()

In [126]:
ndim = 10
mean = (np.random.rand(ndim)-1)*3
bounds=np.ones((ndim, 2))
bounds[:, 0] = -3
bounds[:, 1] = 3
C = np.eye(ndim)
theta = multivariate_trunc_normal(mean, C, bounds)
theta

c:\users\chapm\anaconda3\lib\site-packages\emcee\ensemble.py:335: RuntimeWarning: invalid value encountered in subtract
  lnpdiff = (self.dim - 1.) * np.log(zz) + newlnprob - lnprob0
c:\users\chapm\anaconda3\lib\site-packages\emcee\ensemble.py:336: RuntimeWarning: invalid value encountered in greater
  accept = (lnpdiff > np.log(self._random.rand(len(lnpdiff))))


array([-1.22724161, -2.86914463, -1.65654342, -0.41049168, -1.48898967,
       -2.15420681,  0.19776529, -1.54608222, -0.12784448, -1.19448813])

Now we will have a look at RJMCMC techniques here.

In this notebook, we will consider "AutoRJ" where the parameters can be assumed to follow some kind of random walk and the first and second moments are known. 

In [127]:
mu1 = (np.random.rand(5)-1)*3
C1 = C[:5, :5]

B = cholesky(C)
B1 = cholesky(C1)

# now we sample u1 as standard normal.
u1 = np.random.normal(size=5)

In [128]:
R, _ = qr(B)
rank_part = R.dot(inv(B)).dot((theta - mean).flatten())
theta1 = mu1 + B1.dot(rank_part[:B1.shape[0]])
# this is our new parameters for our model
theta1

array([-2.92024446, -1.79184414, -1.32174186, -1.21175407,  1.01030361])

In [130]:
# if we assume that we will grow to 15 dimensions instead
mu1 = (np.random.rand(15)-1)*3
C1 = np.eye(15)
B1 = cholesky(C1)

# now we sample u1 as standard normal
u1 = np.random.normal(size=15)

In [131]:
R, _ = qr(B1)

inner_part = inv(B).dot((theta - mean).flatten())
rank_part = R.dot(np.hstack([inner_part, mu1[-5:]]))

In [132]:
theta1 = mu1 + B1.dot(rank_part[:B1.shape[0]])
# this is our new parameters for our model
theta1

array([-2.31843724, -1.35249321, -1.56818934, -1.45872298,  0.45455409,
        0.58780382,  0.04155598, -0.18231594, -0.47882022, -3.3333339 ,
       -1.96595753, -0.01140445, -3.14393964, -2.54075202, -4.70696646])